In [1]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt

import sys
sys.path
sys.path.append('../scenes/')
import syndiff as sd 

from scipy.optimize import minimize

In [3]:

    return np.nansum(np.abs(template - ga))


In [2]:
Ra =  336#95.4586
Dec = 47#-51.2377
Size = 20
Maglim= 16
Bkg_limit = 20.5
Zeropoint = 20.44
Scale = 100
Interpolate = False
tpf = sd.Get_TESS(Ra,Dec,Size)
# pos returned as column row 
pos, gmag = sd.Get_Gaia(tpf,magnitude_limit=Maglim)
col = pos[:,0]
row = pos[:,1]

Tmag = gmag - 0.5
tcounts = 10**(-2/5*(Tmag - Zeropoint))
bkg = 10**(-2/5*(Bkg_limit - Zeropoint))

In [ ]:
guess = [0,0]
pos_corr = minimize(Optimise_scene,guess)

In [4]:
global tpf
global pos
global tcounts

In [5]:
def Optimise_scene(Delt):
    """
    row is Delt[0]
    col is Delt[1]
    """
    sources = np.zeros((len(pos),tpf.shape[1],tpf.shape[2]))
    for i in range(len(pos)):
        template = np.zeros((2*tpf.shape[1],2*tpf.shape[2]))
        kernal = sd.Get_PRF(pos[i,0] + tpf.row + Delt[1], pos[i,1] + tpf.column + Delt[0],
                         tpf.camera,tpf.ccd)
        #print(template.shape)
        offset1 = int(0.5 * tpf.shape[1])
        offset2 = int(0.5 * tpf.shape[2])
        obj_row = int(pos[i,1] + offset1 + Delt[0]) 
        obj_col = int(pos[i,0] + offset2+ Delt[1])
        if ((obj_row >= 0) & (obj_row < template.shape[0]) &
           (obj_row >= 0) & (obj_col < template.shape[1])):
            template[obj_row,obj_col] = tcounts[i]
            template = signal.fftconvolve(template, kernal, mode='same')
        template = template[offset1:int(3*offset1),offset2-1:int(3*offset2-1)]
        sources[i] = template #+ 10**(-2/5*(20.5 - 20.44))
    template = np.nanmedian(tpf.flux,axis=0)
    ga = np.nansum(sources,axis=0)
    print(Delt)